In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas import DataFrame

### 获取数据源

In [2]:
user = "meuser"
pwd = "NLb6ceTnOKAILorG8VVdY"
server_logs = "119.23.8.231"
port = "27128"
db_name = "mobility_exchange"

uri = 'mongodb://' + user + ':' + pwd + '@' + server_logs + ':' + port

client = MongoClient(uri)
db = client[db_name]

pipeline = [{"$match": {"logDate": 20180702}},
            {"$project": {"_id":"null","updateTime": 1, "event": 1, 'meterTripId': 1}},
            {"$sort": {"updateTime": 1}}]

data = db["meMogTransportHubEvent"].aggregate(pipeline, allowDiskUse=True)

l = []
for c in data:
    l.append(c)
    
df = DataFrame(l)
df.head()

,_id,event,meterTripId,updateTime
0,null,METER_ON,989841563652096,2000-10-13 00:33:03
1,null,METER_OFF,989860572520448,2018-06-30 20:42:18
2,null,METER_OFF,989860572520448,2018-06-30 20:42:18
3,null,METER_OFF,989860572520448,2018-06-30 20:42:18
4,null,METER_ON,989866461245440,2018-06-30 20:44:02


### 对数据源进行分组处理，获取每一个计价器行程的关键时间和更新时间

In [3]:
del df["_id"]

In [4]:
df.head()

,event,meterTripId,updateTime
0,METER_ON,989841563652096,2000-10-13 00:33:03
1,METER_OFF,989860572520448,2018-06-30 20:42:18
2,METER_OFF,989860572520448,2018-06-30 20:42:18
3,METER_OFF,989860572520448,2018-06-30 20:42:18
4,METER_ON,989866461245440,2018-06-30 20:44:02


In [5]:
r = {}
for index, row in df.iterrows():
    if row["meterTripId"] == "":
        continue
    if row["meterTripId"] in r:
        r[row["meterTripId"]]+=("::"+row["event"])
    else:
        r[row["meterTripId"]]=""
        r[row["meterTripId"]]+=("::"+row["event"])
        
df_r = DataFrame({"ID":list(r.keys()),"events":list(r.values())})   

In [6]:
df_r.head()

,ID,events
0,989841563652096,::METER_ON::METER_OFF
1,989860572520448,::METER_OFF::METER_OFF::METER_OFF::METER_ON
2,989866461245440,::METER_ON::METER_OFF::METER_OFF
3,989784565268480,::METER_OFF::METER_OFF::METER_OFF::METER_ON
4,989778905916416,::METER_OFF::METER_OFF::METER_OFF::METER_ON


In [7]:
out_in_out = "OUT::IN_HIRED::OUT"
flag = []
for index,row in df_r.iterrows():
    if out_in_out in row["events"]:
        flag.append(True)
    else:
        flag.append(False)
df_r["flag"] = flag
df_r.head()

,ID,events,flag
0,989841563652096,::METER_ON::METER_OFF,False
1,989860572520448,::METER_OFF::METER_OFF::METER_OFF::METER_ON,False
2,989866461245440,::METER_ON::METER_OFF::METER_OFF,False
3,989784565268480,::METER_OFF::METER_OFF::METER_OFF::METER_ON,False
4,989778905916416,::METER_OFF::METER_OFF::METER_OFF::METER_ON,False


In [10]:
df_end = df_r[flag]
df_end.to_csv("20180702.csv")